In [1]:
from phishingDetection.urlPishDect.modules.Prediction import PredictionURLS
from phishingDetection.emailPishDect.modules.FeatureExtractionEmail import FeatureExtractionEmail
from flask_cors import CORS
from flask import Flask,render_template, request,jsonify

import warnings
import joblib

In [2]:
def loadPredict(predict):
    
    loaded_model = joblib.load('phishingDetection/emailPishDect/model/best_model.pkl')

    label = loaded_model.predict(predict)

    return label


def predictionEmail(email_content):
        
    obj=FeatureExtractionEmail(email_content)

    df = obj.df

    prad = loadPredict(df)
    
    return prad

In [3]:
def predictionURLS( urls=[]):
    
    warnings.filterwarnings("ignore", category=UserWarning, message="Trying to unpickle estimator.*from version.*when using version.*")  

    obj = PredictionURLS(urls)

    return obj.resultOutput


In [4]:


app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/urlpredict', methods=['POST'])
def urlpredict():
    
    url = request.form.get('url', '')
    
    url_result = predictionURLS([url])

    # Render the result template with the prediction result
    return render_template('index.html', url_result=url_result)


@app.route('/emailpredict', methods=['POST'])
def emailpredict():
    
    email = request.form.get('email', '')
    
    email_result = predictionEmail(email)

    # Render the result template with the prediction result
    return render_template('index.html', email_result=email_result)


@app.route('/urlpredictExt', methods=['POST'])
def urlpredictExt():
    
    data = request.json
    url = data.get('url', '')
    
    url_result = predictionURLS([url])

    return {"result": f"Url Result {url_result}"}

@app.route('/emailpredictExt', methods=['POST'])
def emailpredictExt():
    
    data = request.json
    email = data.get('email', '')
    

    email_result = predictionEmail(email)

    # Render the result template with the prediction result
    return {"result": f"Email Result {email_result}"}



if __name__ == '__main__':
    
    app.run(host='0.0.0.0', port=8080, debug=False, use_reloader=False)




 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.0.102:8080
Press CTRL+C to quit
192.168.0.102 - - [05/Mar/2024 18:36:25] "OPTIONS /urlpredictExt HTTP/1.1" 200 -
192.168.0.102 - - [05/Mar/2024 18:36:30] "POST /urlpredictExt HTTP/1.1" 200 -
192.168.0.102 - - [05/Mar/2024 18:36:31] "OPTIONS /emailpredictExt HTTP/1.1" 200 -
c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
192.168.0.102 - - [05/Mar/2024 18:36:31] "POST /emailpredictExt HTTP/1.1" 200 -


THhis is my email-------------- Delivered-To: awaisamin09@gmail.com
Received: by 2002:a05:6022:7414:b0:4f:17fb:7a83 with SMTP id dp20csp2724789lab;
        Mon, 26 Feb 2024 09:51:44 -0800 (PST)
X-Received: by 2002:a17:90a:6c63:b0:296:f2d5:6567 with SMTP id x90-20020a17090a6c6300b00296f2d56567mr10329070pjj.4.1708969904159;
        Mon, 26 Feb 2024 09:51:44 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1708969904; cv=none;
        d=google.com; s=arc-20160816;
        b=cTcHzHt4ObLfjxxcqi70hzMl5aI7OsA1l4fNy9skuxQPGBO4A82Yqd5OZR6a/m1U6Y
         +Z44cDJaqsgUZfB+szLIuLRVxQW3eYssU1gyu6LyYXCq1p2jhPm3L1sQBijXm3+V0Dy7
         fEOekzumVPk3Ul4jUF92wRN/LIPD4FpaffO7HR3efxoESML/PpFzYoav46sytC4ZK6QK
         58wOib6vjyiQKMK5VDYzXm+sDBxstzG68CUtTnfa5s64E/c+ZxbtpXlhAY6MpXnzqAl1
         +hz6maaHzDxCeqabW+LTQr0XHt+/mME3gx7HvxjP52AuGkAI9LBz2HkGZcoTGYFkgpb5
         4uQQ==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;
        h=to:subject:message-id:date:from:mi

192.168.0.102 - - [05/Mar/2024 18:36:42] "OPTIONS /urlpredictExt HTTP/1.1" 200 -
192.168.0.102 - - [05/Mar/2024 18:36:46] "POST /urlpredictExt HTTP/1.1" 200 -


: 